In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import  pyplot as plt
import missingno as msno
from datetime import date
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.neighbors import LocalOutlierFactor
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, StandardScaler, RobustScaler
from sklearn.linear_model import LinearRegression

In [42]:
df=pd.read_csv('Telco-Customer-Churn.csv')

In [43]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
SeniorCitizen,7043.0,0.162147,0.368612,0.00,0.0,0.00,0.00,1.00
tenure,7043.0,32.371149,24.559481,0.00,9.0,29.00,55.00,72.00
MonthlyCharges,7043.0,64.761692,30.090047,18.25,35.5,70.35,89.85,118.75


In [44]:
df.describe(include='object').T

,count,unique,top,freq
customerID,7043,7043,7590-VHVEG,1
gender,7043,2,Male,3555
Partner,7043,2,No,3641
Dependents,7043,2,No,4933
PhoneService,7043,2,Yes,6361
MultipleLines,7043,3,No,3390
InternetService,7043,3,Fiber optic,3096
OnlineSecurity,7043,3,No,3498
OnlineBackup,7043,3,No,3088
DeviceProtection,7043,3,No,3095


In [45]:
df.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [46]:
df.columns=[col.upper() for col in df.columns]

In [5]:
df.head()

,CUSTOMERID,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,...,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


# Görev 1 : Keşifçi Veri Analizi

In [6]:
# Adım 1: Genel resmi inceleyiniz.
def genraly(data):
    print('##### head #######')
    print(data.head())
    print('##### shape #######')
    print(data.shape)
    print('##### dtypes #######')
    print(data.dtypes)
    print('##### info #######')
    print(data.info())
    print('##### describe #######')
    print(data.describe().T)
    print('##### tail #######')
    print(data.tail())

genraly(df)  

##### head #######
   CUSTOMERID  GENDER  SENIORCITIZEN PARTNER DEPENDENTS  TENURE PHONESERVICE   
0  7590-VHVEG  Female              0     Yes         No       1           No  \
1  5575-GNVDE    Male              0      No         No      34          Yes   
2  3668-QPYBK    Male              0      No         No       2          Yes   
3  7795-CFOCW    Male              0      No         No      45           No   
4  9237-HQITU  Female              0      No         No       2          Yes   

      MULTIPLELINES INTERNETSERVICE ONLINESECURITY  ... DEVICEPROTECTION   
0  No phone service             DSL             No  ...               No  \
1                No             DSL            Yes  ...              Yes   
2                No             DSL            Yes  ...               No   
3  No phone service             DSL            Yes  ...              Yes   
4                No     Fiber optic             No  ...               No   

  TECHSUPPORT STREAMINGTV STREAMINGMOVIES  

In [16]:
# Adım 2: Numerik ve kategorik değişkenleri yakalayınız.
def search(data, cat_th=10, car_th=20):
    cat_cols=[col for col in data.columns if data[col].dtypes =='O']
    num_but_cols=[col for col in data.columns if data[col].nunique() < cat_th and data[col].dtypes !='O']
    #car_but_cat=[col for col in data.columns if data[col].nunique() > car_th and data[col].dtypes=='O']
    cat_cols=cat_cols+num_but_cols
    #cat_cols=[col for col in cat_cols if col not in car_but_cat]

    
    num_cols=[col for col in data.columns if data[col].dtypes !='O']
    #num_cols=[col for col in num_col if col not in num_but_cols]
    
    print(f'gozlem sayi: {data.shape[0]}')
    print(f'deyisken sayi: {data.shape[1]}')
    print(f'categorik: {len(cat_cols)}')
    print(f'number gibi gorunen: {len(num_but_cols)}')
   # print(f'cardinal: {len(car_but_cat)}')
    print(f'sayisal deyisken :{len(num_cols)}')
    
    
    
    return cat_cols,num_but_cols,num_cols


    

In [17]:
cat_cols,num_but_cols, num_cols=search(df)

gozlem sayi: 7043
deyisken sayi: 21
categorik: 19
number gibi gorunen: 1
sayisal deyisken :3


In [18]:
# Adım 3.1: Numerik  değişkenlerin analizini yapınız.
def number(data,num_cols):
    quantiles=[0.05,0.10,0.25,0.30,0.40,0.50,0.60,0.70,0.80,0.90,0.95,0.99]
    print(data[num_cols].describe(quantiles))

In [19]:
for col in num_cols:
    number(df,col)

count    7043.000000
mean        0.162147
std         0.368612
min         0.000000
5%          0.000000
10%         0.000000
25%         0.000000
30%         0.000000
40%         0.000000
50%         0.000000
60%         0.000000
70%         0.000000
80%         0.000000
90%         1.000000
95%         1.000000
99%         1.000000
max         1.000000
Name: SeniorCitizen, dtype: float64
count    7043.000000
mean       32.371149
std        24.559481
min         0.000000
5%          1.000000
10%         2.000000
25%         9.000000
30%        12.000000
40%        20.000000
50%        29.000000
60%        40.000000
70%        50.000000
80%        60.000000
90%        69.000000
95%        72.000000
99%        72.000000
max        72.000000
Name: tenure, dtype: float64
count    7043.000000
mean       64.761692
std        30.090047
min        18.250000
5%         19.650000
10%        20.050000
25%        35.500000
30%        45.850000
40%        58.830000
50%        70.350000
60%        

In [11]:
# kategorik değişkenlerin analizini yapınız.
def category(data, col_name):
    print(pd.DataFrame({col_name: data[col_name].value_counts(),
                       'Ratio': 100 * data[col_name].value_counts() / len(data)}))

In [12]:
for col in cat_cols:
    category(df,col)

            CUSTOMERID     Ratio
CUSTOMERID                      
7590-VHVEG           1  0.014198
3791-LGQCY           1  0.014198
6008-NAIXK           1  0.014198
5956-YHHRX           1  0.014198
5365-LLFYV           1  0.014198
...                ...       ...
9796-MVYXX           1  0.014198
2637-FKFSY           1  0.014198
1552-AAGRX           1  0.014198
4304-TSPVK           1  0.014198
3186-AJIEK           1  0.014198

[7043 rows x 2 columns]
        GENDER     Ratio
GENDER                  
Male      3555  50.47565
Female    3488  49.52435
         PARTNER     Ratio
PARTNER                   
No          3641  51.69672
Yes         3402  48.30328
            DEPENDENTS      Ratio
DEPENDENTS                       
No                4933  70.041176
Yes               2110  29.958824
              PHONESERVICE      Ratio
PHONESERVICE                         
Yes                   6361  90.316626
No                     682   9.683374
                  MULTIPLELINES      Ratio
MULTIPL

In [13]:
# Adım 4: Hedef değişken analizi yapınız. (Kategorik değişkenlere göre hedef değişkenin ortalaması, hedef değişkene görenumerik değişkenlerin ortalaması)
def hedef(data,target,numbers):
    print(pd.DataFrame(data.groupby(target).agg({numbers:['mean','std','sum']})))

In [14]:
for col in num_cols:
    hedef(df,'CUSTOMERID',col)

           SENIORCITIZEN        
                    mean std sum
CUSTOMERID                      
0002-ORFBO           0.0 NaN   0
0003-MKNFE           0.0 NaN   0
0004-TLHLJ           0.0 NaN   0
0011-IGKFF           1.0 NaN   1
0013-EXCHZ           1.0 NaN   1
...                  ...  ..  ..
9987-LUTYD           0.0 NaN   0
9992-RRAMN           0.0 NaN   0
9992-UJOEL           0.0 NaN   0
9993-LHIEB           0.0 NaN   0
9995-HOTOH           0.0 NaN   0

[7043 rows x 3 columns]
           TENURE        
             mean std sum
CUSTOMERID               
0002-ORFBO    9.0 NaN   9
0003-MKNFE    9.0 NaN   9
0004-TLHLJ    4.0 NaN   4
0011-IGKFF   13.0 NaN  13
0013-EXCHZ    3.0 NaN   3
...           ...  ..  ..
9987-LUTYD   13.0 NaN  13
9992-RRAMN   22.0 NaN  22
9992-UJOEL    2.0 NaN   2
9993-LHIEB   67.0 NaN  67
9995-HOTOH   63.0 NaN  63

[7043 rows x 3 columns]
           MONTHLYCHARGES           
                     mean std    sum
CUSTOMERID                          
0002-ORFBO   

In [15]:
# Adım 5: Aykırı gözlem analizi yapınız.
def ayk(data,col_names, q1=0.25,q3=0.75):
    query1=data[col_names].quantile(q1)
    queyr3=data[col_names].quantile(q3)
    iqr=queyr3 - query1
    low_l=query1 - 1.5 * iqr
    up_l=queyr3 + 1.5 *iqr
    return low_l,up_l

In [16]:
for col in num_cols:
    print(col,ayk(df,col))

SENIORCITIZEN (0.0, 0.0)
TENURE (-60.0, 124.0)
MONTHLYCHARGES (-46.02499999999999, 171.375)


In [17]:
def chech(data,col_name):
    low_l,up_l=ayk(data,col_name)
    if data[(data[col_name] < low_l) | (data[col_name] > up_l)].any(axis=None):
        return True
    else:
        return False

In [18]:
for col in num_cols:
    print(col,chech(df,col))

SENIORCITIZEN True
TENURE False
MONTHLYCHARGES False


In [19]:
def grab_out(data,col_name):
    low_l,up_l=ayk(data,col_name)
    if data[((data[col_name] < low_l) | (data[col_name] > up_l))].shape[0] > 10:
        print(data[((data[col_name] < low_l) | (data[col_name] > up_l))].head())
    else:
         print(data[((data[col_name] < low_l)| (data[col_name] > up_l))])

In [20]:
for col in num_cols:
    grab_out(df,col)

    CUSTOMERID  GENDER  SENIORCITIZEN PARTNER DEPENDENTS  TENURE PHONESERVICE   
20  8779-QRDMV    Male              1      No         No       1           No  \
30  3841-NFECX  Female              1     Yes         No      71          Yes   
31  4929-XIHVW    Male              1     Yes         No       2          Yes   
34  3413-BMNZE    Male              1      No         No       1          Yes   
50  8012-SOUDQ  Female              1      No         No      43          Yes   

       MULTIPLELINES INTERNETSERVICE ONLINESECURITY  ... DEVICEPROTECTION   
20  No phone service             DSL             No  ...              Yes  \
30               Yes     Fiber optic            Yes  ...              Yes   
31                No     Fiber optic             No  ...              Yes   
34                No             DSL             No  ...               No   
50               Yes     Fiber optic             No  ...               No   

   TECHSUPPORT STREAMINGTV STREAMINGMOVIES        

In [21]:
def replace(data,col_name):
    low_l,up_l=ayk(data,col_name)
    data.loc[(data[col_name]<low_l)]=low_l
    data.loc[(data[col_name] > up_l)]=up_l
    return low_l,up_l

In [22]:
for col in num_cols:
    print(replace(df,col))

(0.0, 0.0)
(-70.0, 122.0)
(-75.47500000000001, 179.125)


In [23]:
for col in num_cols:
    print(col,chech(df,col))

SENIORCITIZEN False
TENURE False
MONTHLYCHARGES False


In [24]:
# Adım 6: Eksik gözlem analizi yapınız.
def missing (data):
    miss=[col for col in data.columns if data[col].isnull().sum() > 0]
    na_sort=data[miss].isnull().sum().sort_values(ascending=False)
    ratio=(data[miss].isnull().sum()/ data.shape[0] * 100)
    concat=pd.concat([na_sort, np.round(ratio,2)], axis=1, keys=['missing_values','ratio'])
    print(concat)

In [25]:
missing(df)

Empty DataFrame
Columns: [missing_values, ratio]
Index: []


In [26]:
df.head()

,CUSTOMERID,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,...,DEVICEPROTECTION,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [27]:
# Adım 2: Yeni değişkenler oluşturunuz
df.loc[(df['GENDER']=='Male') & (df['SENIORCITIZEN']==0),'New_Gender']='Young_male'
df.loc[(df['GENDER']=='Male') & (df['SENIORCITIZEN']==1),'New_Gender']='Senior_male'

In [28]:
df.head()

,CUSTOMERID,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,...,TECHSUPPORT,STREAMINGTV,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN,New_Gender
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,NaN
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,No,No,One year,No,Mailed check,56.95,1889.5,No,Young_male
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,Young_male
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,Young_male
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,NaN


In [29]:
df.loc[(df['GENDER']=='Female') & (df['SENIORCITIZEN']==0), 'New_Gender']='Young_female'
df.loc[(df['GENDER']=='Female') & (df['SENIORCITIZEN']==1), 'New_Gender']='Senior_female'

In [30]:
df['New_Gender'].value_counts()

New_Gender
Young_male      2981
Young_female    2920
Name: count, dtype: int64

In [31]:
df.groupby('GENDER').agg({'TENURE':'mean'})

,TENURE
GENDER,
0.0,0.000000
Female,32.171233
Male,32.212680


In [32]:
df.loc[(df['TENURE'] >= 0) & (df['TENURE'] < 30 ), 'New_Tenure']='silver_customer'
df.loc[(df['TENURE'] >= 30) & (df['TENURE'] < 50),'New_Tenure']='gold_customer'
df.loc[(df['TENURE'] >= 50) & (df['TENURE'] < 72), 'New_Tenure']='premium_customer'

In [33]:
df['New_Tenure'].value_counts()

New_Tenure
silver_customer     4157
premium_customer    1504
gold_customer       1076
Name: count, dtype: int64

In [34]:
df.loc[(df['GENDER']=='Male') & (df['PHONESERVICE']=='Yes'), 'Phone_servcice']='Yes_male_phone_sercice'
df.loc[(df['GENDER']=='Male') & (df['PHONESERVICE']=='No'), 'Phone_servcice']='No_male_phone_sercice'

In [35]:
df.loc[(df['GENDER']=='Female') & (df['PHONESERVICE']=='Yes'), 'Phone_servcice']='Yes_famele_phone_service'
df.loc[(df['GENDER']=='Female') & (df['PHONESERVICE']=='No'),'Phone_servcice']='No_female_phone_service'

In [36]:
df.head(15)

,CUSTOMERID,GENDER,SENIORCITIZEN,PARTNER,DEPENDENTS,TENURE,PHONESERVICE,MULTIPLELINES,INTERNETSERVICE,ONLINESECURITY,...,STREAMINGMOVIES,CONTRACT,PAPERLESSBILLING,PAYMENTMETHOD,MONTHLYCHARGES,TOTALCHARGES,CHURN,New_Gender,New_Tenure,Phone_servcice
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,Month-to-month,Yes,Electronic check,29.85,29.85,No,Young_female,silver_customer,No_female_phone_service
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,No,One year,No,Mailed check,56.95,1889.5,No,Young_male,gold_customer,Yes_male_phone_sercice
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes,Young_male,silver_customer,Yes_male_phone_sercice
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,No,One year,No,Bank transfer (automatic),42.30,1840.75,No,Young_male,gold_customer,No_male_phone_sercice
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes,Young_female,silver_customer,Yes_famele_phone_service
5,9305-CDSKC,Female,0,No,No,8,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,99.65,820.5,Yes,Young_female,silver_customer,Yes_famele_phone_service
6,1452-KIOVK,Male,0,No,Yes,22,Yes,Yes,Fiber optic,No,...,No,Month-to-month,Yes,Credit card (automatic),89.10,1949.4,No,Young_male,silver_customer,Yes_male_phone_sercice
7,6713-OKOMC,Female,0,No,No,10,No,No phone service,DSL,Yes,...,No,Month-to-month,No,Mailed check,29.75,301.9,No,Young_female,silver_customer,No_female_phone_service
8,7892-POOKP,Female,0,Yes,No,28,Yes,Yes,Fiber optic,No,...,Yes,Month-to-month,Yes,Electronic check,104.80,3046.05,Yes,Young_female,silver_customer,Yes_famele_phone_service
9,6388-TABGU,Male,0,No,Yes,62,Yes,No,DSL,Yes,...,No,One year,No,Bank transfer (automatic),56.15,3487.95,No,Young_male,premium_customer,Yes_male_phone_sercice


In [37]:
df.loc[(df['GENDER']=='Male') & (df['SENIORCITIZEN']==0) & (df['PARTNER']=='Yes'), 'New_Partner']='Male_Partner'
df.loc[(df['GENDER']=='Male') & (df['SENIORCITIZEN']==1) & (df['PARTNER']=='No'), 'New_Partner']='Male_Partner'

In [38]:
df['New_Gender'].value_counts()

New_Gender
Young_male      2981
Young_female    2920
Name: count, dtype: int64

df.head(100)

In [47]:
# Adım 3: Encoding işlemlerini gerçekleştiriniz.
def one_hot_encoder(data, col_ns, drop_first=True):
    data=pd.get_dummies(data, columns=col_ns, drop_first=drop_first)
    return data